In [1]:
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAI

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "secrets/glossy-attic-415618-93704b0714e2.json"

In [3]:
model = VertexAI(model_name="gemini-pro")

In [4]:
llm = ChatVertexAI(model_name="gemini-pro",project_id="glossy-attic-415618")


In [5]:
#llm.invoke("who are you?")

#### LangChain Guide
https://medium.com/google-cloud/question-a-web-page-using-vertex-ai-palm-api-langchain-and-streamlit-8b56f0753cec

In [1]:
#import libraries
import vertexai
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings

import streamlit as st
import requests
from bs4 import BeautifulSoup

from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
# gets model
llm = VertexAI(
    model_name="gemini-pro",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,)

embeddings = VertexAIEmbeddings()

/home/alex/anaconda3/envs/gcptest/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAI` was deprecated in LangChain 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(
/home/alex/anaconda3/envs/gcptest/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAIEmbeddings` was deprecated in LangChain 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAIEmbeddings`.
  warn_deprecated(
Model_name will become a required arg for Vertex

In [8]:
### Gets text from webpage
def get_text(url):
   # Send a GET request to the URL
   response = requests.get(url)

   # Create a BeautifulSoup object with the HTML content
   soup = BeautifulSoup(response.content, "html.parser")

   # Find the specific element or elements containing the text you want to scrape
   # Here, we'll find all <p> tags and extract their text
   paragraphs = soup.find_all("p")

   # Loop through the paragraphs and print their text
   with open("text\\temp.txt", "w", encoding='utf-8') as file:
       # Loop through the paragraphs and write their text to the file
       for paragraph in paragraphs:
           file.write(paragraph.get_text() + "\n")

In [9]:
### caches unserializable objects that you dont want t oload multiple times
# saves things that cannot be stored in a database
@st.cache_resource
def create_langchain_index(input_text):
    print("--indexing---")
    get_text(input_text)
    loader = TextLoader("text\\temp.txt", encoding='utf-8')
    # data = loader.load()

    index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch,
              embedding=embeddings).from_loaders([loader])
    # using vertex ai embeddings initialized above
    return index

In [10]:
### checks if function called with same paramteres and code
### saves data that is structured and can be out in a database
@st.cache_data
def get_response(input_text,query):
    print(f"--querying---{query}")
    response = index.query(query,llm=llm)
    return response

2024-04-23 16:39:34.838 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [11]:
input_text = "https://apnews.com/article/illicit-marijuana-operation-charges-maine-ad17b4bf2cab5cf4379fac394045335b"

In [13]:
index = create_langchain_index(input_text)
summary_query ="Write a 100 words summary of the document"
summary_response = get_response(input_text,summary_query)

tweet_query ="Write a twitter tweet and add hashtags"
tweet_response =  get_response(input_text,tweet_query)

ln_query ="Write a linkedin post for the document"
ln_response = get_response(input_text,ln_query)

--indexing---


/home/alex/anaconda3/envs/gcptest/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
2024-04-23 16:40:37.785 No runtime found, using MemoryCacheStorageManager


--querying---Write a 100 words summary of the document
--querying---Write a twitter tweet and add hashtags
--querying---Write a linkedin post for the document


In [31]:
summary_response

'## Summary of the Document:\n\nThis article reports on the increasing problem of illegal marijuana grow operations in Maine, with a focus on a recent case involving a Chinese national. The article highlights the challenges posed by these operations, which often involve sophisticated setups and connections to international trafficking networks. Law enforcement agencies are working to combat these operations, using tactics such as monitoring utility bills and collaborating across jurisdictions. The article concludes by emphasizing the need for continued vigilance and collaboration to address this issue. \n'

In [32]:
tweet_response

'## Maine authorities bust illicit marijuana grow operation run by Chinese national \n\n**40 pounds of processed marijuana seized, investigation into international criminal organization continues.** #Maine #Marijuana #IllicitGrow #DEA #FBI #OrganizedCrime \n\n**Source:** AP News \n'

In [33]:
ln_response

'## Maine authorities shut down illegal marijuana grow operation with international ties\n\n**A Chinese citizen has been arrested in Maine for allegedly running a high-tech, illicit marijuana grow operation in a rural home.** The operation was discovered after law enforcement officials noticed a significant increase in the home\'s electricity usage. \n\n**According to court documents, the home\'s electric capability was doubled, and the monthly electricity bill went from $300 to nearly $9,000.** This, along with other factors, led authorities to suspect illegal activity. \n\n**The operation was raided in February, and 40 pounds of processed marijuana were seized.** The home owner, a limited liability company, has not yet been charged. Two other individuals who were present at the time of the raid were released without charges.\n\n**This case highlights the growing problem of illegal marijuana grow operations with international connections.** Law enforcement officials are concerned that

### Working with secret

In [2]:
import json

In [5]:
#json.load(open("secrets/glossy-attic-415618-93704b0714e2.json"))